# Torch-ONNX-TensorRT Workflow example

This Jupyter notebook serves as a guide to understand how the workflow implemented in the experiments of this work functions. The workflow itself is described in the following image:

![Workflow.](/outputs/img_readme/TensorRT_workflow.png)

In this case, we will use the described workflow in a specific example: image classification using the ImageNet-1k dataset with the MobileNetV2 model.

## Base model

To obtain the base model, we use the pre-trained models on ImageNet-1k provided by PyTorch:

In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.15.2', "mobilenet_v2", weights=f'MobileNet_V2_Weights.DEFAULT')

We define some constants that will be used by our model throughout the workflow:

In [ ]:
# CONSTANTES
BATCH_SIZE = 1
C = 3 # number of channels of the input image
H = 224 # heigh of the input image
W = 224 # width of the input image
NETWORK = 'mobilenet' # mobiletv2
current_directory = os.getcwd()

## ONNX: Model Conversion

To convert the model from `.pt` to `.onnx`, we use the code described in `onnx_transform.py` as follows:

In [ ]:
%run onnx_transform.py --weights weights/best.pth --pretrained --network $NETWORK --input_shape $BATCH_SIZE $C $H $W

This has created the ONNX model, saved in `weights/best.onn`. Now, we need to create the TensorRT Application and optimize the model with it.

## TensorRT Application

To create the TensorRT Application and optimize the model in ONNX format, we use the code in build_trt.py. This code calls `utils/engine.py`, which describes the TensorRT Application. In `utils/engine.py`, you can make further changes to experiment with different types of optimizations.

### TRT fp32

In [ ]:
%run ./build_trt.py --weights weights/best.onnx  --fp32 --input_shape $BATCH_SIZE $C $H $W --engine_name best_fp32.engine

### TRT fp16

In [ ]:
%run build_trt.py --weights weights/best.onnx  --fp16 --input_shape $BATCH_SIZE $C $H $W --engine_name best_fp16.engine

### TRT int8

In [ ]:
import subprocess
subprocess.Popen('rm -r outputs/cache > /dev/null 2>&1', shell=True)
%run build_trt.py --weights weights/best.onnx  --int8 --input_shape $BATCH_SIZE $C $H $W --engine_name best_int8.engine

For more specific optimizations, refer to the wiki... (a wiki will be available soon)

The optimized models are stored in the `weights` folder.

## Running Phase: Inference using optimized model

First, we ensure that the system has a GPU capable of using CUDA. Then, we load the optimized models and deserialize them:

In [ ]:
import os
from utils.engine import TRTModule

gpu_available = torch.cuda.is_available()
if not gpu_available:
    print('CUDA is not available.')
else:
    print('CUDA is available.')

device = torch.device("cuda:0" if gpu_available else "cpu")

engine_path_1 = os.path.join(current_directory,"weights/best_fp32.engine")
engine_path_2 = os.path.join(current_directory,"weights/best_fp16.engine")
engine_path_3 = os.path.join(current_directory,"weights/best_int8.engine")
Engine_fp32 = TRTModule(engine_path_1,device)
Engine_fp16 = TRTModule(engine_path_2,device)
Engine_int8 = TRTModule(engine_path_3,device)
Engine_fp32.set_desired(['outputs'])
Engine_fp16.set_desired(['outputs'])
Engine_int8.set_desired(['outputs'])

We load the ImageNet-1k validation dataset, available [here](https://drive.google.com/drive/folders/1xHxI_S03Wjh56g2W8_1pNzXCTiKheqYY?usp=sharing):

In [ ]:
from utils.data_loader import val_data_loader
val_loader = val_data_loader(os.path.join(current_directory,'datasets/dataset_val/val'), batch_size=BATCH_SIZE, workers=4, pin_memory=False)

We validate the models using the validate function described below:

In [ ]:
from utils.helper import AverageMeter, accuracy
import time
import torch.nn as nn

def validate(model_version, val_loader, model, criterion=nn.CrossEntropyLoss().to(device)):
    batch_time_all = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    model.to(device)
    model.eval()

    # Calculate 10% of total batches
    warmup_batches = int(0.1 * len(val_loader))
    
    # Initialize the maximum and minimum processing time after warm-up
    max_time_all = 0
    min_time_all = float('inf')

    num_batches_to_process = int(1 * len(val_loader))

    for i, (input, target) in enumerate(val_loader):
        if i >= num_batches_to_process:
            break

        target = target.to(device)
        start_all = time.time() # start time, moving data to gpu
        input = input.to(device)
        
        with torch.no_grad():
            output = model(input)
            output_cpu = output.cpu() # needed to get the time from gpu to cpu
            all_time = (time.time() - start_all) * 1000  # Convert to milliseconds / time when the result pass to cpu again 
            loss = criterion(output, target)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.item(), input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time in milliseconds and ignore first 10% batches
        if i >= warmup_batches:
            batch_time_all.update(all_time)
            max_time_all = max(max_time_all, all_time)
            min_time_all = min(min_time_all, all_time)
       
    print("|  Model          | Latency avg (ms)| Latency max (ms) | accuracy (Prec@1) (%)|accuracy (Prec@5) (%)|")
    print("|-----------------|-----------------|------------------|----------------------|---------------------|")
    print("| {:<15} | {:<15.1f} | {:<15.1f} |  {:<20.2f} | {:<20.2f} | ".format(
        model_version,
        batch_time_all.avg, max_time_all,
        top1.avg, top5.avg))
    return

### Base Model Validation

In [ ]:
validate('Base Model', val_loader, model)

### TRT fp32 Validation

In [ ]:
validate('TRT fp32', val_loader, Engine_fp32)

### TRT fp16 Validation

In [ ]:
validate('TRT fp32', val_loader, Engine_fp16)

### TRT int8 Validation

In [ ]:
validate('TRT fp32', val_loader, Engine_int8)

For the specific usage of `build_trt.py` and `onnx_transform.py`, refer to the wiki.

To generate the tables presented with metrics such as latency, throughput, parameters, weights, etc., refer to the `experiments/main` folder.